# Getting started


**To run the app you will also need to access to ColabPro (because of High RAM option)**

In [ ]:
!rm -rf TaskMatrix && git clone https://github.com/microsoft/TaskMatrix

Cloning into 'TaskMatrix'...
remote: Enumerating objects: 560, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 560 (delta 23), reused 51 (delta 16), pack-reused 493
Receiving objects: 100% (560/560), 40.26 MiB | 12.45 MiB/s, done.
Resolving deltas: 100% (192/192), done.


In [ ]:
cd TaskMatrix/

/content/TaskMatrix


**Install packages**


In [ ]:
!pip install -r requirements.txt
!pip install git+https://github.com/IDEA-Research/GroundingDINO.git
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.0/344.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 73.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.4/229.4 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 107.5 MB/s eta 0:00:00
     ━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/IDEA-Research/GroundingDINO.git to /tmp/pip-req-build-scxio7cm
  Running command git clone --filter=blob:none --quiet https://github.com/IDEA-Research/GroundingDINO.git /tmp/pip-req-build-scxio7cm
  Resolved https://github.com/IDEA-Research/GroundingDINO.git to commit 31aa788a3cf6ab82d27385f39e242b34af9cc011
  Preparing metadata (setup.py) ... done
  Created wheel for groundingdino: filename=groundingdino-0.1.0-cp310-cp310-linux_x86_64.whl size=2987214 sha256=84a35e5b0aa4c79ab66741284c068f324af781ca1da1f1eabbdecb90c5f840a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-l8bvqu_h/wheels/6b/06/d7/b57f601a4df56af41d262a5b1b496359b13c323bf5ef0434b2
Successfully built groundingdino
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-bu

**Set your OPENAI_API_KEY**


In [ ]:
%env OPENAI_API_KEY=OPENAI_API_KEY

# Run VisualChatGPT

In [ ]:
from visual_chatgpt import *

In [ ]:
!mkdir -p checkpoints
load = "Text2Box_cuda:0,Segmenting_cuda:0,Inpainting_cuda:0,ImageCaptioning_cuda:0"

load_dict = {e.split('_')[0].strip(): e.split('_')[1].strip() for e in load.split(',')}
bot = ConversationBot(load_dict=load_dict)
with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    lang = gr.Radio(choices = ['Chinese','English'], value=None, label='Language')
    chatbot = gr.Chatbot(elem_id="chatbot", label="Visual ChatGPT")
    state = gr.State([])
    with gr.Row(visible=False) as input_raws:
        with gr.Column(scale=0.7):
            txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter, or upload an image").style(
                container=False)
        with gr.Column(scale=0.15, min_width=0):
            clear = gr.Button("Clear️")
        with gr.Column(scale=0.15, min_width=0):
            btn = gr.UploadButton("🖼️", file_types=["image"])
    lang.change(bot.init_agent, [lang], [input_raws, lang, txt, clear])
    txt.submit(bot.run_text, [txt, state], [chatbot, state])
    txt.submit(lambda: "", None, txt)
    btn.upload(bot.run_image, [btn, state, txt, lang], [chatbot, state, txt])
    clear.click(bot.memory.clear)
    clear.click(lambda: [], None, chatbot)
    clear.click(lambda: [], None, state)
    demo.launch(server_name="0.0.0.0", server_port=7860, share=True, debug=True)

Initializing VisualChatGPT, load_dict={'Text2Box': 'cuda:0', 'Segmenting': 'cuda:0', 'Inpainting': 'cuda:0', 'ImageCaptioning': 'cuda:0'}
Initializing ObjectDetection to cuda:0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
Inintializing Segmentation to cuda:0


...

======>Auto Resize Image...
Resize image form 1200x800 to 512x320


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



Processed ImageCaptioning, Input Image: image/b846becf.png, Output Text: a group of people sitting around a table with laptops

Processed run_image, Input image: image/b846becf.png
Current state: [('![](file=image/b846becf.png)*image/b846becf.png*', 'Received.  ')]
Current Memory: 
Human: provide a figure named image/b846becf.png. The description is: a group of people sitting around a table with laptops. This information helps you to understand this image, but you should use tools to finish following tasks, rather than directly imagine from my description. If you understand, say "Received". 
AI: Received.  
history_memory:
Human: provide a figure named image/b846becf.png. The description is: a group of people sitting around a table with laptops. This information helps you to understand this image, but you should use tools to finish following tasks, rather than directly imagine from my description. If you understand, say "Received". 
AI: Received.  , n_tokens: 50


> Entering new Agent

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Processed ObejectSegmenting, Input Image: image/46bc5145.png, Object to be Segment  seed, Output Image: image/c238_segmentation_46bc5145_46bc5145.png

Observation: image/c238_segmentation_46bc5145_46bc5145.png
Thought:Action: Get Photo Description
Action Input: image/c238_segmentation_46bc5145_46bc5145.png
Processed ImageCaptioning, Input Image: image/c238_segmentation_46bc5145_46bc5145.png, Output Text: a close up of a sliced avocai on a wooden table

Observation: a close up of a sliced avocai on a wooden table
Thought:AI: The seed of the avocados in the image/46bc5145.png is brown.

> Finished chain.

Processed run_text, Input text: image/46bc5145.png what color is its seed?
Current state: [('![](file=image/46bc5145.png)*image/46bc5145.png*', 'Received.  '), (' image/46bc5145.png what do you see?', 'I see avocados with leaves on a wooden cutting board in the ![](file=image/46bc5145.png)*image/46bc5145.png*.'), ('image/46bc5145.png what color is its seed?', 'The seed of the avocados 